In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.metrics import Precision, Recall
import os
import pandas as pd 
import numpy as np
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
import matplotlib as plt

In [2]:
image_dir = "archive-7"

In [3]:
def create_dataframe(dir):
    image_location = []
    labels = []
    for label in os.listdir(dir):
        label_path = os.path.join(dir, label)
        if os.path.isdir(label_path):
            labels.extend([label] * len(os.listdir(label_path)))
            for image in os.listdir(label_path):
                image_location.append(os.path.join(label_path, image))
            print(label, "completed")
        else:
            print(f"{label} is not a folder, skipping...")
    return image_location, labels

In [4]:
data = pd.DataFrame()
data['image'], data['label'] = create_dataframe(image_dir)
data.to_markdown()

labels.csv is not a folder, skipping...
happy completed
contempt completed
sad completed
fear completed
surprise completed
neutral completed
anger completed
disgust completed


In [17]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale=True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

In [18]:
train_features = extract_features(train['image'])
validation_features = extract_features(validation['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


  0%|          | 0/7066 [00:00<?, ?it/s]

In [20]:
x_train = train_features /225.0
x_val = validation_features / 225.0

In [22]:
label_encoder = LabelEncoder()
label_encoder.fit(train['label'])
y_train = label_encoder.transform(train['label'])
y_val = label_encoder.transform(validation['label'])
y_train = to_categorical(y_train, num_classes=7)
y_val = to_categorical(y_val, num_classes=7)

In [33]:
# sequential model
model = Sequential()

# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# flattening
model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(7, activation='softmax'))
model.summary()
# model compilation
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy', Precision(), Recall()])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 46, 46, 128)       1280      
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 23, 23, 128)       0         
 ng2D)                                                           
                                                                 
 dropout_24 (Dropout)        (None, 23, 23, 128)       0         
                                                                 
 conv2d_17 (Conv2D)          (None, 21, 21, 256)       295168    
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 10, 10, 256)       0         
 ng2D)                                                           
                                                                 
 dropout_25 (Dropout)        (None, 10, 10, 256)      

In [34]:
history = model.fit(x=x_train,
          y=y_train,
          batch_size=128,
          epochs=100,
          validation_data=(x_val, y_val)
)

Epoch 1/100


2024-03-17 21:32:27.168635: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


226/226 [==============================] - ETA: 0s - loss: 1.8341 - accuracy: 0.2377 - precision_2: 0.2424 - recall_2: 0.0011

2024-03-17 21:33:22.375233: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


226/226 [==============================] - 59s 245ms/step - loss: 1.8341 - accuracy: 0.2377 - precision_2: 0.2424 - recall_2: 0.0011 - val_loss: 1.8139 - val_accuracy: 0.2583 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 2/100
226/226 [==============================] - 62s 273ms/step - loss: 1.8120 - accuracy: 0.2482 - precision_2: 0.2683 - recall_2: 3.8167e-04 - val_loss: 1.7957 - val_accuracy: 0.2654 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 3/100
226/226 [==============================] - 56s 248ms/step - loss: 1.7810 - accuracy: 0.2632 - precision_2: 0.5169 - recall_2: 0.0096 - val_loss: 1.6871 - val_accuracy: 0.3140 - val_precision_2: 0.7644 - val_recall_2: 0.0243
Epoch 4/100
226/226 [==============================] - 74s 325ms/step - loss: 1.7760 - accuracy: 0.3015 - precision_2: 0.4965 - recall_2: 0.0680 - val_loss: 1.7522 - val_accuracy: 0.3191 - val_precision_2: 0.5571 - val_recall_2: 0.0401
Epoch 5/100
226/226 [===========================

In [ ]:
history_dict = history.history
acc = history_dict['accuracy']
loss = history_dict['loss']
precision = history_dict['precision']
recall = history_dict['recall']
val_acc = history.history.get('val_accuracy')
val_loss = history.history.get('val_loss')

epochs = range(1, len(acc) + 1)
plt.figure(figsize=(12,8))
plt.subplot(2,2,1)
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title("Training and Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['Accuracy', 'Val. Accuracy'])


plt.subplot(2,2,2)
plt.plot(epochs,loss)
plt.plot(epochs,val_loss)
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(['Loss', 'Val. Loss'])
plt.tight_layout()
plt.show()